## Getting Started with Langchain with Open AI

In this quickstart with Langchain, LangSmith and LangServe

- Get setup with LangChain, LangSmith and LangServe
- Use the most baisc and common componenets of Langchain : prompt templates, models, and output parsers.
- Build a simple application with Langchain
- Trace your appllication with LangSmith
- Serve your application with LangServe

### Important Components of Langchain

1. RAG - Retrieval Augmented Generation
    1. Intial starting phase
        1. Load Data Source - can be multiple files (Data Ingestion)
        2. Split the Data to smaller chunks (Data Transformation)
        3. Convert the chucks data to vectors (Embedding)
        4. Storing the vector data (VectorStore DB)
    2. Second phase
        1. We give a prompt template (Like which gerne)
        2. Question will be inputed by the person
        3. it will go through vector db
            1. Retrival Chain - Interface for VectorDB
        4. Goes through the LLM
        5. Output given to the Person

### Data Ingestion in Langchain

https://python.langchain.com/v0.2/docs/integrations/document_loaders/


In [1]:
## Text Loader

from langchain_community.document_loaders import TextLoader

loader = TextLoader('Files/speech.txt')

loader

In [2]:
text_document = loader.load()
text_document

[Document(metadata={'source': 'Files/speech.txt'}, page_content='Ladies and gentlemen, esteemed guests, and fellow gamers,\n\nToday, we gather to celebrate a momentous occasion in the world of gaming—a historic collaboration that brings together two iconic franchises loved by millions across the globe. It is with great excitement and anticipation that I present to you the groundbreaking partnership between Mario and FIFA, resulting in a new game that promises to redefine the boundaries of sports and adventure.\n\nFor decades, Mario, the beloved plumber from the Mushroom Kingdom, has captured our hearts with his timeless adventures, creative gameplay, and indomitable spirit. From rescuing Princess Peach to racing through exhilarating tracks in Mario Kart, Mario has become a symbol of fun, creativity, and the joy of gaming.\n\nSimilarly, FIFA, the unrivaled titan of football simulation, has provided countless hours of thrilling matches, strategic gameplay, and an authentic experience of 

In [6]:
## PDF Loader

from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader('Files/MindSearch.pdf')
loader

pdf_document = loader.load()
pdf_document

[Document(metadata={'source': 'Files/MindSearch.pdf', 'page': 0}, page_content='Technical Report\nMindSearch 思·索:\nMimicking Human Minds Elicits Deep AI Searcher\nZehui Chen1∗, Kuikun Liu2∗, Qiuchen Wang1, Jiangning Liu2,\nWenwei Zhang2, Kai Chen2†, Feng Zhao1†\n1University of Science and Technology of China\n2Shanghai AI Laboratory\nAbstract\nInformation seeking and integration is a complex cognitive task that consumes enor-\nmous time and effort. Search engines reshape the way of seeking information but often\nfail to align with complex human intentions. Inspired by the remarkable progress of\nLarge Language Models (LLMs), recent works attempt to solve the information-seeking\nand integration task by combining LLMs and search engines. However, these methods\nstill obtain unsatisfying performance due to three challenges: (1) complex requests often\ncannot be accurately and completely retrieved by the search engine once; (2) corre-\nsponding information to be integrated is spread over 

In [8]:
type(pdf_document[0])

langchain_core.documents.base.Document

In [16]:
## Web based Loader

from langchain_community.document_loaders import WebBaseLoader
import bs4

loader = WebBaseLoader(web_paths = ('https://www.indiatoday.in/sports/olympics/story/paris-olympics-vinesh-phogat-comeback-story-timeline-khota-sikka-gold-standard-2578161-2024-08-07',),bs_kwargs=dict(parse_only=bs4.SoupStrainer(class_ = ("post-title","post-content","post-header"))))

loader.load()


[Document(metadata={'source': 'https://www.indiatoday.in/sports/olympics/story/paris-olympics-vinesh-phogat-comeback-story-timeline-khota-sikka-gold-standard-2578161-2024-08-07'}, page_content='')]

In [18]:
## Arxiv Loader

from langchain_community.document_loaders import ArxivLoader

doc = ArxivLoader(query = "1605.08386",load_max_docs = 2).load()

doc

[Document(metadata={'Published': '2016-05-26', 'Title': 'Heat-bath random walks with Markov bases', 'Authors': 'Caprice Stanley, Tobias Windisch', 'Summary': 'Graphs on lattice points are studied whose edges come from a finite set of\nallowed moves of arbitrary length. We show that the diameter of these graphs on\nfibers of a fixed integer matrix can be bounded from above by a constant. We\nthen study the mixing behaviour of heat-bath random walks on these graphs. We\nalso state explicit conditions on the set of moves so that the heat-bath random\nwalk, a generalization of the Glauber dynamics, is an expander in fixed\ndimension.'}, page_content='arXiv:1605.08386v1  [math.CO]  26 May 2016\nHEAT-BATH RANDOM WALKS WITH MARKOV BASES\nCAPRICE STANLEY AND TOBIAS WINDISCH\nAbstract. Graphs on lattice points are studied whose edges come from a ﬁnite set of\nallowed moves of arbitrary length. We show that the diameter of these graphs on ﬁbers of a\nﬁxed integer matrix can be bounded from above

In [20]:
## Wikipedia loader

from langchain_community.document_loaders import WikipediaLoader

docs = WikipediaLoader(query="GenAI",load_max_docs = 2).load()
len(docs)

docs

[Document(metadata={'title': 'Generative artificial intelligence', 'summary': 'Generative artificial intelligence (generative AI, GenAI, or GAI) is artificial intelligence capable of generating text, images, videos, or other data using generative models, often in response to prompts. Generative AI models learn the patterns and structure of their input training data and then generate new data that has similar characteristics.\nImprovements in transformer-based deep neural networks, particularly large language models (LLMs), enabled an AI boom of generative AI systems in the early 2020s. These include chatbots such as ChatGPT, Copilot, Gemini and LLaMA, text-to-image artificial intelligence image generation systems such as Stable Diffusion, Midjourney and DALL-E, and text-to-video AI generators such as Sora. Companies such as OpenAI, Anthropic, Microsoft, Google, and Baidu as well as numerous smaller firms have developed generative AI models.\nGenerative AI has uses across a wide range o

### Text Splitting from Documents

In [1]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader('Files/MindSearch.pdf')

docs = loader.load()

docs

[Document(metadata={'source': 'Files/MindSearch.pdf', 'page': 0}, page_content='Technical Report\nMindSearch 思·索:\nMimicking Human Minds Elicits Deep AI Searcher\nZehui Chen1∗, Kuikun Liu2∗, Qiuchen Wang1, Jiangning Liu2,\nWenwei Zhang2, Kai Chen2†, Feng Zhao1†\n1University of Science and Technology of China\n2Shanghai AI Laboratory\nAbstract\nInformation seeking and integration is a complex cognitive task that consumes enor-\nmous time and effort. Search engines reshape the way of seeking information but often\nfail to align with complex human intentions. Inspired by the remarkable progress of\nLarge Language Models (LLMs), recent works attempt to solve the information-seeking\nand integration task by combining LLMs and search engines. However, these methods\nstill obtain unsatisfying performance due to three challenges: (1) complex requests often\ncannot be accurately and completely retrieved by the search engine once; (2) corre-\nsponding information to be integrated is spread over 

1. Recursive Character Text Splitter

In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 50)
## final_document = text_splitter.create_documents(docs)  for this to happen the type of the loader should be text not documents
final_document = text_splitter.split_documents(docs)

final_document

[Document(metadata={'source': 'Files/MindSearch.pdf', 'page': 0}, page_content='Technical Report\nMindSearch 思·索:\nMimicking Human Minds Elicits Deep AI Searcher\nZehui Chen1∗, Kuikun Liu2∗, Qiuchen Wang1, Jiangning Liu2,\nWenwei Zhang2, Kai Chen2†, Feng Zhao1†\n1University of Science and Technology of China\n2Shanghai AI Laboratory\nAbstract\nInformation seeking and integration is a complex cognitive task that consumes enor-\nmous time and effort. Search engines reshape the way of seeking information but often'),
 Document(metadata={'source': 'Files/MindSearch.pdf', 'page': 0}, page_content='fail to align with complex human intentions. Inspired by the remarkable progress of\nLarge Language Models (LLMs), recent works attempt to solve the information-seeking\nand integration task by combining LLMs and search engines. However, these methods\nstill obtain unsatisfying performance due to three challenges: (1) complex requests often\ncannot be accurately and completely retrieved by the sea

In [17]:
## Spliting text files using Recursive CHaracter Text Splitter

from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

docs = TextLoader('Files/speech.txt').load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 100, chunk_overlap = 20)
final_document = text_splitter.create_documents([docs])

final_document


TypeError: expected string or bytes-like object, got 'list'

2. Character Text splitter

In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter

docs = TextLoader('Files/speech.txt').load()
text_splitter = CharacterTextSplitter(separator="\n\n",chunk_size=100,chunk_overlap=20)
final_document = text_splitter.split_documents(docs)

final_document

Created a chunk of size 389, which is longer than the specified 100
Created a chunk of size 314, which is longer than the specified 100
Created a chunk of size 327, which is longer than the specified 100
Created a chunk of size 437, which is longer than the specified 100
Created a chunk of size 356, which is longer than the specified 100
Created a chunk of size 327, which is longer than the specified 100
Created a chunk of size 417, which is longer than the specified 100
Created a chunk of size 310, which is longer than the specified 100
Created a chunk of size 332, which is longer than the specified 100


[Document(metadata={'source': 'Files/speech.txt'}, page_content='Ladies and gentlemen, esteemed guests, and fellow gamers,'),
 Document(metadata={'source': 'Files/speech.txt'}, page_content='Today, we gather to celebrate a momentous occasion in the world of gaming—a historic collaboration that brings together two iconic franchises loved by millions across the globe. It is with great excitement and anticipation that I present to you the groundbreaking partnership between Mario and FIFA, resulting in a new game that promises to redefine the boundaries of sports and adventure.'),
 Document(metadata={'source': 'Files/speech.txt'}, page_content='For decades, Mario, the beloved plumber from the Mushroom Kingdom, has captured our hearts with his timeless adventures, creative gameplay, and indomitable spirit. From rescuing Princess Peach to racing through exhilarating tracks in Mario Kart, Mario has become a symbol of fun, creativity, and the joy of gaming.'),
 Document(metadata={'source': 'Fi

3. HTML Header Text Splitter

In [3]:
from langchain_text_splitters import HTMLHeaderTextSplitter

html_string = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Example Page with Headers</title>
</head>
<body>
    <h1>Main Header</h1>
    <p>This is the main section of the page.</p>

    <h2>Sub Header 1</h2>
    <p>This is the first subsection of the page.</p>

    <h3>Sub Header 2</h3>
    <p>This is the second subsection of the page.</p>
</body>
</html>
"""

header_to_split_on = [
    ("h1","Header 1"),
    ("h2","Header 2"),
    ("h3","Header 3")
]

html_splitter = HTMLHeaderTextSplitter(header_to_split_on)
html_header_splits = html_splitter.split_text(html_string)

html_header_splits

[Document(metadata={'Header 1': 'Main Header'}, page_content='This is the main section of the page.'),
 Document(metadata={'Header 1': 'Main Header', 'Header 2': 'Sub Header 1'}, page_content='This is the first subsection of the page.'),
 Document(metadata={'Header 1': 'Main Header', 'Header 2': 'Sub Header 1', 'Header 3': 'Sub Header 2'}, page_content='This is the second subsection of the page.')]

In [4]:
url = "https://www.indiatoday.in/india/story/union-minister-kiren-rijiju-waqf-bill-lok-sabha-opposition-protest-2578988-2024-08-08"

header_to_split_on = [
    ("h1","Header 1"),
    ("h2","Header 2"),
    ("h3","Header 3"),
    ("h4","Header 4")
]

html_splitter = HTMLHeaderTextSplitter(header_to_split_on)
html_header_splits = html_splitter.split_text_from_url(url)

html_header_splits


[Document(page_content="India TodayAaj TakGNTTVLallantopBusiness TodayBanglaMalayalamNortheastBT BazaarHarper's BazaarSports TakCrime TakAstro TakGamingBrides TodayCosmopolitanKisan TakIshq FMIndia Today HindiReader’s DigestAaj Tak Campus  \nIndia TodayAaj TakGNTTVLallantopBusiness TodayBanglaMalayalamNortheastBT BazaarHarper's BazaarSports Tak  \nMagazine  \nLive TV  \nSearch  \nSEARCH  \nSIGN IN  \nEdition  \nIN  \nINUS  \nHome  \nTV  \nLive TVPrimetime  \nMagazine  \nLatest EditionInsightBest Colleges  \nLife+Style  \nIndia  \nSouth  \nWorld  \nBusiness  \nAll Sports  \nSports TodayOlympics 2024Medal TallyOlympics ScheduleCricketFootballTennis  \nTechnology  \nEntertainment  \nShowbuzzBollywoodHollywoodOTTLatest Reviews  \nSpecials  \nPodcastsFirst Things FastSunday SpecialHistory of ItNewsMoDIUInteractivesOpinionGames  \nVideos  \nFact Check  \nOther News  \nEducationIt's ViralScienceHealthAutoLaw TodayEnvironmentCitiesWeatherWeb StoriesHoroscopes  \nFollow Us On:  \nNewsIndia'Waqf

4. Recursive Json Splitter

In [7]:
import json 
import requests
from langchain_text_splitters import RecursiveJsonSplitter

json_data = requests.get("https://api.smith.langchain.com/openapi.json").json()
json_splitter = RecursiveJsonSplitter(max_chunk_size=300)
json_chunks = json_splitter.split_json(json_data)

json_chunks[0]


{'openapi': '3.1.0',
 'info': {'title': 'LangSmith', 'version': '0.1.0'},
 'paths': {'/api/v1/sessions/{session_id}': {'get': {'tags': ['tracer-sessions'],
    'summary': 'Read Tracer Session',
    'description': 'Get a specific session.'}}}}

In [11]:
## The splitter can also output documents 

docs = json_splitter.create_documents(texts=[json_data])
texts = json_splitter.split_text(json_data)

docs
texts

['{"openapi": "3.1.0", "info": {"title": "LangSmith", "version": "0.1.0"}, "paths": {"/api/v1/sessions/{session_id}": {"get": {"tags": ["tracer-sessions"], "summary": "Read Tracer Session", "description": "Get a specific session."}}}}',
 '{"paths": {"/api/v1/sessions/{session_id}": {"get": {"operationId": "read_tracer_session_api_v1_sessions__session_id__get", "security": [{"API Key": []}, {"Tenant ID": []}, {"Bearer Auth": []}]}}}}',
 '{"paths": {"/api/v1/sessions/{session_id}": {"get": {"parameters": [{"name": "session_id", "in": "path", "required": true, "schema": {"type": "string", "format": "uuid", "title": "Session Id"}}, {"name": "include_stats", "in": "query", "required": false, "schema": {"type": "boolean", "default": false, "title": "Include Stats"}}, {"name": "accept", "in": "header", "required": false, "schema": {"anyOf": [{"type": "string"}, {"type": "null"}], "title": "Accept"}}]}}}}',
 '{"paths": {"/api/v1/sessions/{session_id}": {"get": {"responses": {"200": {"descripti

### Embeddings

1. OpenAI Embeddings

In [13]:
import os
from dotenv import load_dotenv
load_dotenv()  #load all the environment variables


True

In [15]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [16]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
embeddings

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x7f8732d3a480>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x7f8732d3a1e0>, model='text-embedding-3-large', dimensions=None, deployment='text-embedding-ada-002', openai_api_version='', openai_api_base=None, openai_api_type='', openai_proxy='', embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [ ]:
text = "this is a tutorial on OpenAI embeddings"
query_result = embeddings.embed_query(text)
query_result

In [ ]:
len(query_result) ## basically the abovery text is converted to vector with standard dimention of 3702

In [ ]:
embeddings_1024 = OpenAIEmbeddings(model="text-embedding-3-large",dimensions=1024)

In [ ]:
text = "this is a tutorial on OpenAI embeddings"
query_result = embeddings_1024.embed_query(text)
query_result

In [ ]:
## for a whole document 

from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma 

docs = TextLoader('Files/MindSearch.pdf').load
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 100, chunk_overlap = 20)
final_document = text_splitter.split_documents(docs)

db = Chroma.from_documents(final_document,embeddings_1024)
db


In [ ]:
query = "It will be a;; easier for us to conduct oursleves as belligerants"
retrived_results = db.similarity(query)
print(retrived_results)

2. Ollama Embeddings

Ollama supports embdding models, making it possible to build retrival augemented generation (RAG) applications that combine text prompts with existing documents or other data.

In [27]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings = (
    OllamaEmbeddings(model="gemma2:2b")  #by default is llama 2
)

r1 = embeddings.embed_documents(       ## embedings of documents
    [
        "Teja is good",
        "Rishi is also good"
    ]
)

r2 = embeddings.embed_query("Baba Black sheep")   ## embedding of qyery


r2

[-4.433842182159424,
 0.15139240026474,
 -0.12527352571487427,
 3.0743048191070557,
 1.4821760654449463,
 0.7764095067977905,
 -2.3502728939056396,
 2.4110653400421143,
 1.4507063627243042,
 3.3687286376953125,
 -0.5011345148086548,
 -1.3026353120803833,
 -2.1421573162078857,
 0.5402129292488098,
 0.799353837966919,
 0.28028756380081177,
 -2.889907121658325,
 -0.04419819638133049,
 -9.063619613647461,
 1.1271575689315796,
 1.1184436082839966,
 -3.2639598846435547,
 -1.9920154809951782,
 -0.43995577096939087,
 1.018789529800415,
 0.9690299034118652,
 -0.47340887784957886,
 -0.601116955280304,
 -0.6220418810844421,
 -0.26346680521965027,
 -1.1325652599334717,
 -0.4134972095489502,
 0.335091233253479,
 -1.7488090991973877,
 0.4521881341934204,
 -0.7555142045021057,
 -0.2807672619819641,
 -0.3304063379764557,
 1.4170891046524048,
 -1.3113574981689453,
 1.7215782403945923,
 1.4659050703048706,
 1.4759801626205444,
 2.2155139446258545,
 -0.38576915860176086,
 -1.5156638622283936,
 1.05907070

In [29]:
### Other EMmbedding Models

embeddings = OllamaEmbeddings(model="mxbai-embed-large")
text = "Lol Big dawg"
query_result = embeddings.embed_query(text)

query_result

[1.0740042924880981,
 -0.4888085126876831,
 0.2179730385541916,
 0.7262692451477051,
 -0.3553701639175415,
 -0.25298529863357544,
 0.5191358327865601,
 0.8254109025001526,
 0.0511750727891922,
 0.9541998505592346,
 0.21359875798225403,
 0.5939348936080933,
 -0.18747663497924805,
 0.13180264830589294,
 -0.4073277413845062,
 0.4633234441280365,
 -0.4057236611843109,
 -0.7223130464553833,
 -0.5504478216171265,
 0.29196470975875854,
 0.2871347963809967,
 0.16177630424499512,
 -1.2742223739624023,
 0.5047949552536011,
 0.3121003806591034,
 0.3488137423992157,
 -0.6227443218231201,
 0.42444390058517456,
 1.2829744815826416,
 0.32024192810058594,
 -0.18297091126441956,
 -0.22565984725952148,
 -0.2514530420303345,
 -0.8492094874382019,
 0.027868792414665222,
 -1.1975733041763306,
 0.46449267864227295,
 -0.27152466773986816,
 -0.8622336387634277,
 -0.4227551519870758,
 -0.3276945650577545,
 -0.3788226842880249,
 0.5722383856773376,
 -0.4984908699989319,
 -0.8651925325393677,
 0.326665461063385,

3. Hugging Face Embeddings 

In [30]:
import os
from dotenv import load_dotenv
load_dotenv()  #load all the environment variables

os.environ['HUGGINGFACE_API_KEY'] = os.getenv("HUGGINGFACE_API_KEY")


Sentence Transformers on huggingface

HuggingFace sentence-transformers is a python framework for state of the art sentence, text and image embeddings. One of the embedding models is used in the HuggingFaceEmbeddings class. We have also added an alias for Sentence Transformer Embeddings for user who are more familar with directly using that package.

In [32]:
from langchain_huggingface import HuggingFaceEmbeddings

HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


/home/ntejha/anaconda3/envs/yoda/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [35]:
text = "This is my text document"

query_result = embeddings.embed_query(text)

query_result

[0.1359676718711853,
 -0.7294342517852783,
 0.6610684394836426,
 0.25944235920906067,
 0.42489826679229736,
 0.15432099997997284,
 -0.1439814567565918,
 0.05289699137210846,
 1.0109176635742188,
 0.8399329781532288,
 -0.3657636046409607,
 0.5664397478103638,
 0.15212169289588928,
 0.29013487696647644,
 -0.9227873682975769,
 -0.3759389817714691,
 -0.26453864574432373,
 -0.1906420886516571,
 -0.6858856081962585,
 0.19197317957878113,
 -0.5492956042289734,
 0.4977536201477051,
 -1.7977311611175537,
 0.16736756265163422,
 0.35084599256515503,
 -0.1589636504650116,
 -0.47102564573287964,
 -0.1657412052154541,
 0.6783977150917053,
 1.0408579111099243,
 -0.40225890278816223,
 0.598585307598114,
 -0.3758746385574341,
 -0.2140234410762787,
 0.19143447279930115,
 -0.22852128744125366,
 0.7220228314399719,
 -0.30011287331581116,
 -0.0069588348269462585,
 -0.7478269934654236,
 -0.26020705699920654,
 0.12704025208950043,
 0.5065754055976868,
 -0.87828129529953,
 -1.0119450092315674,
 -0.04980961605

### Storing of the vector conversions

1. FAISS

Facebook AI Similiarity Search (FIASS) is a library for efficent similarity search and clustering of dense vectors. It contains algorithms that search in sets of vectors of any size, up to ones that possibliy do not fit in RAN. It also cotains supporting code for evaluation and parameter tuning.

In [42]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings

docs = TextLoader('Files/speech.txt').load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100,chunk_overlap = 20)
final_document = text_splitter.split_documents(docs)

embeddings = OllamaEmbeddings(model = "gemma2:2b")

db = FAISS.from_documents(final_document,embeddings)


In [46]:
## query

query = "which to games are joining together?"

docs = db.similarity_search(query)

docs[0]

Document(metadata={'source': 'Files/speech.txt'}, page_content='new generations of gamers, and remind us all why we fell in love with gaming in the first place.')

As a Retriver 

We can also convert the vectorstore into a Retriver class. This allows us to easily use it in other LangChain methods, which largely work with retrivers

In [48]:
retriever = db.as_retriever()
retriever.invoke(query)
docs[0].page_content

'new generations of gamers, and remind us all why we fell in love with gaming in the first place.'

Similarity Search with score

There are some FAISS specific methods. One of them is similarity_search_with_score, which allows you to return not only the documents but also the distance score of the query to them. The returned distance score L2 distance. Therfore, a lower score is better.

In [49]:
docs_and_score = db.similarity_search_with_score(query)
docs_and_score

[(Document(metadata={'source': 'Files/speech.txt'}, page_content='new generations of gamers, and remind us all why we fell in love with gaming in the first place.'),
  4285.048),
 (Document(metadata={'source': 'Files/speech.txt'}, page_content='something that transcends generations and brings joy to players of all ages.'),
  4459.1855),
 (Document(metadata={'source': 'Files/speech.txt'}, page_content='and immersive gameplay, FIFA has united football fans and gamers in a shared passion for the sport.'),
  4488.744),
 (Document(metadata={'source': 'Files/speech.txt'}, page_content='and Princess Peach dazzling us with her skills.'),
  4793.752)]

In [52]:
embedding_vector = embeddings.embed_query(query)
embedding_vector


[-0.7153050303459167,
 0.8315085172653198,
 -0.5437251329421997,
 1.4972516298294067,
 0.310796320438385,
 2.7345616817474365,
 -0.5729087591171265,
 -1.1933681964874268,
 -0.9007325172424316,
 0.27294981479644775,
 0.23559561371803284,
 0.8586082458496094,
 -1.2224023342132568,
 0.12925297021865845,
 -0.3571043312549591,
 -2.0779545307159424,
 1.2879046201705933,
 -0.05317963659763336,
 -2.7708282470703125,
 0.3609040081501007,
 -0.14678430557250977,
 -1.2808488607406616,
 -1.0520436763763428,
 0.5342895984649658,
 0.08075238019227982,
 -1.2269313335418701,
 -0.40321728587150574,
 -0.8493734002113342,
 -0.9008727669715881,
 -0.92702317237854,
 0.9384509325027466,
 -0.7595142722129822,
 -2.3992440700531006,
 1.9604990482330322,
 1.3968425989151,
 -0.026150720193982124,
 -0.8131150007247925,
 -0.21793580055236816,
 0.9340750575065613,
 -1.9109904766082764,
 0.6239103674888611,
 2.5435874462127686,
 0.7857377529144287,
 -1.7070302963256836,
 1.1019591093063354,
 -2.1584978103637695,
 -0.

In [55]:
docs_score = db.similarity_search_by_vector(embedding_vector)
docs_score

[Document(metadata={'source': 'Files/speech.txt'}, page_content='new generations of gamers, and remind us all why we fell in love with gaming in the first place.'),
 Document(metadata={'source': 'Files/speech.txt'}, page_content='something that transcends generations and brings joy to players of all ages.'),
 Document(metadata={'source': 'Files/speech.txt'}, page_content='and immersive gameplay, FIFA has united football fans and gamers in a shared passion for the sport.'),
 Document(metadata={'source': 'Files/speech.txt'}, page_content='and Princess Peach dazzling us with her skills.')]

In [59]:
## Saving and Loading 

db.save_local('faiss_index')

new_df = FAISS.load_local('faiss_index',embeddings, allow_dangerous_deserialization=True)

2. Chroma

Chroma is a AI-native open-source vector database focused on developer productivity and happiess. Chroma is licensed under Apache 2.0.

In [64]:
## buliding a sample vector db

from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

docs = TextLoader('Files/speech.txt').load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 100, chunk_overlap = 20)
splits = text_splitter.split_documents(docs)

embeddings = OllamaEmbeddings(model = "gemma2:2b")
vectordb = Chroma.from_documents(splits, embeddings)
vectordb


In [66]:
## Query it

query = "Mario is which character"

query_result = vectordb.similarity_search(query)

query_result[0].page_content

'collaboration that brings together two iconic franchises loved by millions across the globe. It is'

In [74]:
##Saving and loading

vectordb = Chroma.from_documents(splits, embeddings, persist_directory='./chroma_db')
db2 = Chroma(persist_directory='./chroma_db',embedding_function=embeddings)

doc_res = db2.similarity_search(query)
print(doc_res[0].page_content)

docs_res = db2.similarity_search_with_score(query)
print(doc_res[0].page_content)

## Reteriver option

retriver = vectordb.as_retriever()
retriever.invoke(query)[0].page_content

collaboration that brings together two iconic franchises loved by millions across the globe. It is
collaboration that brings together two iconic franchises loved by millions across the globe. It is


'collaboration that brings together two iconic franchises loved by millions across the globe. It is'

### Building Important Components of Langchain

In [5]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
## Langsmith Tracking
os.environ['LANGCHAIN_API_KEY'] = os.getenv("LANGCHAIN_API_KEY")
os.environ['LANGCHAIN_TRACKING_V2'] = "True"
os.environ['LANGCHAIN_PROJECT'] = os.getenv("LANGCHAIN_PROJECT")

In [7]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')

print(llm)

client=<openai.resources.chat.completions.Completions object at 0x7fc9def01700> async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7fc9def140b0> model_name='gpt-4o' openai_api_key=SecretStr('**********') openai_proxy=''


In [ ]:
## Input and get response from LLM

results = llm.invoke("What is generative AI?")
print(results)

## Once any thing invoked, the Langsmith tracks all your invokings


In [12]:
### Prompt Template

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system","Your an expert AI Engineer. Provide me answers based on the question"),
        ("user","[input]")
    ]
)

prompt

ChatPromptTemplate(input_variables=[], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Your an expert AI Engineer. Provide me answers based on the question')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='[input]'))])

In [ ]:
## Chain

chain = prompt|llm
response = chain.invoke(["input":"Can you tell me about Langchain?"])
print(response)
type(response)

## Stringoutput Parser : How do u want the llm to display the message

from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
chain = prompt|llm|output_parser

response = chain.invoke(["input":"Can you tell me about Langchain?"])
print(response)